# log utils

In [ ]:
#| default_exp utils_log

In [ ]:
#| export
from __future__ import annotations
from typing import Optional, Dict, Any, Callable, List
from functools import wraps
from contextvars import ContextVar
import logging
import asyncio
import inspect
from datetime import datetime
from queue import Queue
import json

## Tenant Context

Thread-safe context variables for tenant_id, user_id, and request_id that propagate across async boundaries.

In [ ]:
#| export
# Context variables for multi-tenant logging
_tenant_id: ContextVar[Optional[str]] = ContextVar('tenant_id', default=None)
_user_id: ContextVar[Optional[str]] = ContextVar('user_id', default=None)
_request_id: ContextVar[Optional[str]] = ContextVar('request_id', default=None)


class TenantContext:
    """Context manager for setting tenant/user/request context
    
    Example:
        with TenantContext(tenant_id='tenant_123', user_id='user_456'):
            # All logging within this block includes tenant/user context
            some_operation()
    """
    
    def __init__(
        self,
        tenant_id: Optional[str] = None,
        user_id: Optional[str] = None,
        request_id: Optional[str] = None
    ):
        self.tenant_id = tenant_id
        self.user_id = user_id
        self.request_id = request_id
        self.tokens = []
    
    def __enter__(self):
        if self.tenant_id:
            self.tokens.append(_tenant_id.set(self.tenant_id))
        if self.user_id:
            self.tokens.append(_user_id.set(self.user_id))
        if self.request_id:
            self.tokens.append(_request_id.set(self.request_id))
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        for token in self.tokens:
            token.var.reset(token)


def get_context() -> Dict[str, Optional[str]]:
    """Get current context variables"""
    return {
        'tenant_id': _tenant_id.get(),
        'user_id': _user_id.get(),
        'request_id': _request_id.get()
    }

In [ ]:
show_doc(get_context)

In [ ]:
show_doc(TenantContext.__init__)

In [ ]:
from nbdev.showdoc import show_doc

## Database Handler

Queue-based database handler for non-blocking log writes. Mock-friendly design for testing.

In [ ]:
#| export
class DatabaseHandler:
    """Non-blocking database log handler with queue
    
    Collects log records in a queue for batch processing.
    Mock-friendly: use write_log() directly in tests.
    """
    
    def __init__(self, db_connection=None, batch_size: int = 20):
        """Initialize database handler
        
        Args:
            db_connection: Database connection (optional for testing)
            batch_size: Number of records to batch before writing
        """
        self.db = db_connection
        self.batch_size = batch_size
        self.queue: Queue = Queue()
        self.logs: List[Dict[str, Any]] = []  # For testing/inspection
    
    def write_log(
        self,
        level: str,
        message: str,
        operation: Optional[str] = None,
        status: str = 'info',
        **kwargs
    ):
        """Write log record to database
        
        Args:
            level: Log level (INFO, ERROR, WARNING)
            message: Log message
            operation: Operation name (for db_operation logs)
            status: Operation status (success, error, info)
            **kwargs: Additional fields (tenant_id, user_id, etc.)
        """
        ctx = get_context()
        
        log_record = {
            'timestamp': datetime.utcnow().isoformat(),
            'level': level,
            'message': message,
            'operation': operation,
            'status': status,
            'tenant_id': ctx['tenant_id'],
            'user_id': ctx['user_id'],
            'request_id': ctx['request_id'],
            **kwargs
        }
        
        # Store for testing/inspection
        self.logs.append(log_record)
        
        # Queue for batch processing (real implementation would process in thread)
        self.queue.put(log_record)
        
        # In real implementation, background thread would batch-write to DB
        # For now, just collect in logs list for testing
    
    def get_logs(self, level: Optional[str] = None) -> List[Dict[str, Any]]:
        """Get collected logs (for testing)"""
        if level:
            return [log for log in self.logs if log['level'] == level]
        return self.logs
    
    def clear_logs(self):
        """Clear logs (for testing)"""
        self.logs.clear()
        while not self.queue.empty():
            self.queue.get()

In [ ]:
show_doc(DatabaseHandler.clear_logs)

In [ ]:
show_doc(DatabaseHandler.get_logs)

In [ ]:
show_doc(DatabaseHandler.write_log)

## FastHTML Integration

Use `@app.before` to set TenantContext once per request:

```python
from fasthtml.common import *
from fh_saasutils_log import TenantContext, DatabaseHandler
import uuid

app = FastHTML()
log_handler = DatabaseHandler(get_host_db())

@app.before
def setup_logging_context(request, session):
    """Set tenant context for every request"""
    tenant_id = session.get('tenant_id')  # From auth
    user_id = session.get('user_id')
    request_id = request.headers.get('X-Request-ID', str(uuid.uuid4()))
    
    ctx = TenantContext(tenant_id, user_id, request_id)
    ctx.__enter__()
    request.state.tenant_ctx = ctx
    request.state.log_handler = log_handler

@app.after
def cleanup_logging_context(request, response):
    """Clean up context after request completes"""
    if hasattr(request.state, 'tenant_ctx'):
        request.state.tenant_ctx.__exit__(None, None, None)
    return response

# Routes automatically have context
@app.get('/users')
async def get_users(request):
    users = await fetch_users()  # Logs include tenant_id
    return users
```

## Logging Decorators

Clean decorator wrappers for automatic logging without cluttering utility functions.

In [ ]:
#| export
def _log_sync_operation(
    func: Callable,
    handler: Optional[DatabaseHandler],
    success_msg: str,
    error_msg: str,
    log_fields: Dict[str, Any],
    operation_name: Optional[str] = None
):
    """Helper for sync operation logging"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = datetime.utcnow()
        op_name = operation_name or func.__name__
        try:
            result = func(*args, **kwargs)
            
            if handler:
                duration_ms = (datetime.utcnow() - start_time).total_seconds() * 1000
                handler.write_log(
                    level='INFO',
                    message=success_msg.format(op_name),
                    operation=op_name,
                    status='success',
                    duration_ms=duration_ms,
                    **log_fields
                )
            
            return result
        
        except Exception as e:
            if handler:
                duration_ms = (datetime.utcnow() - start_time).total_seconds() * 1000
                handler.write_log(
                    level='ERROR',
                    message=error_msg.format(op_name),
                    operation=op_name,
                    status='error',
                    error_type=type(e).__name__,
                    error_message=str(e),
                    duration_ms=duration_ms,
                    **log_fields
                )
            raise
    
    return wrapper


def _log_async_operation(
    func: Callable,
    handler: Optional[DatabaseHandler],
    success_msg: str,
    error_msg: str,
    log_fields: Dict[str, Any],
    operation_name: Optional[str] = None
):
    """Helper for async operation logging"""
    @wraps(func)
    async def wrapper(*args, **kwargs):
        start_time = datetime.utcnow()
        op_name = operation_name or func.__name__
        try:
            result = await func(*args, **kwargs)
            
            if handler:
                duration_ms = (datetime.utcnow() - start_time).total_seconds() * 1000
                handler.write_log(
                    level='INFO',
                    message=success_msg.format(op_name),
                    operation=op_name,
                    status='success',
                    duration_ms=duration_ms,
                    **log_fields
                )
            
            return result
        
        except Exception as e:
            if handler:
                duration_ms = (datetime.utcnow() - start_time).total_seconds() * 1000
                handler.write_log(
                    level='ERROR',
                    message=error_msg.format(op_name),
                    operation=op_name,
                    status='error',
                    error_type=type(e).__name__,
                    error_message=str(e),
                    duration_ms=duration_ms,
                    **log_fields
                )
            raise
    
    return wrapper

In [ ]:
#| export
def log_db_operation(
    operation_type: str = 'QUERY',
    handler: Optional[DatabaseHandler] = None
):
    """Decorator for logging database operations
    
    Args:
        operation_type: Type of operation (SELECT, INSERT, UPDATE, DELETE)
        handler: Database handler (if None, uses global handler)
    
    Example:
        @log_db_operation('INSERT', handler=log_handler)
        def create_user(conn, user_id, email):
            conn.execute("INSERT INTO users...")
            return user_id
    """
    def decorator(func: Callable):
        success_msg = f'{operation_type} operation completed: {{}}'
        error_msg = f'{operation_type} operation failed: {{}}'
        log_fields = {'operation_type': operation_type}
        
        if inspect.iscoroutinefunction(func):
            return _log_async_operation(func, handler, success_msg, error_msg, log_fields)
        else:
            return _log_sync_operation(func, handler, success_msg, error_msg, log_fields)
    
    return decorator

In [ ]:
show_doc(log_db_operation)

In [ ]:
#| export
def log_api_call(
    method: str = 'GET',
    handler: Optional[DatabaseHandler] = None
):
    """Decorator for logging API calls
    
    Args:
        method: HTTP method (GET, POST, PUT, DELETE)
        handler: Database handler (if None, uses global handler)
    
    Example:
        @log_api_call('POST', handler=log_handler)
        async def create_user_endpoint(request):
            return {'status': 'created'}
    """
    def decorator(func: Callable):
        success_msg = f'{method} {{}} completed'
        error_msg = f'{method} {{}} failed'
        log_fields = {'http_method': method}
        
        if inspect.iscoroutinefunction(func):
            return _log_async_operation(func, handler, success_msg, error_msg, log_fields)
        else:
            return _log_sync_operation(func, handler, success_msg, error_msg, log_fields)
    
    return decorator

In [ ]:
show_doc(log_api_call)

In [ ]:
#| export
def log_background_task(
    task_name: str,
    handler: Optional[DatabaseHandler] = None
):
    """Decorator for logging background tasks
    
    Args:
        task_name: Name of background task (e.g. 'data_import', 'email_batch')
        handler: Database handler (if None, uses global handler)
    
    Example:
        @log_background_task('data_import', handler=log_handler)
        async def import_csv(file_path):
            return {'rows_imported': 1000}
    """
    def decorator(func: Callable):
        success_msg = f'Background task {task_name} completed'
        error_msg = f'Background task {task_name} failed'
        log_fields = {'task_type': 'background'}
        
        if inspect.iscoroutinefunction(func):
            return _log_async_operation(func, handler, success_msg, error_msg, log_fields, operation_name=task_name)
        else:
            return _log_sync_operation(func, handler, success_msg, error_msg, log_fields, operation_name=task_name)
    
    return decorator

In [ ]:
show_doc(log_background_task)